# Create dataset from public data at industry level

This file is part of the replication code for: 

Offshore Profit Shifting and Aggregate Measurement: Balance of Payments, Foreign Investment, Productivity, and the Labor Share by Guvenen, Mataloni Jr., Rassier, and Ruhl. 

This version: February 21, 2022

This file processes publicly available data files (located in `1-raw-data`) to create `industry.csv` which is saved in the `3-intermediate-files` folder.

Industries are grouped by their R&D intensity, whether or not they produce IT goods (ITU) and whether or not they use IT goods (ITU).

In [7]:
import pandas as pd
import numpy as np

### RD/ITP/ITU Codes

`industry-classifications.csv` is created by the authors. It assigns industries into differenct industry groups. 

In [8]:
icodes = pd.read_csv('../1-raw-data/author-created-concordances/industry-classifications.csv', 
                     converters={'Description':str.strip})
icodes.head(1)

,RD,NRD,ITU,NITU,ITP,NITP,IED Code,Description
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private industries


## Functions to read in data files

In [9]:
def proc_old_industry(fname, icodes, ftype):
    df = pd.read_csv(fname, header=4, nrows=98, na_values=['...'])
    df = df.rename(columns={'Unnamed: 1': 'Var'})
    df['Var'] = df['Var'].str.strip()

    # We need to modify the descriptions a bit to get them to match up
    recode = {'Publishing industries, except internet (includes software)':'Publishing industries (includes software)',
          'Data processing, internet publishing, and other information services':'Information and data processing services'}
    df['Var'] = df['Var'].replace(recode)
    
    df = pd.merge(left=icodes, right=df, left_on='Description', right_on='Var', how='outer', indicator=True)
    # If the industry does not have a code (RD/ITP/ITU) then we do not need it.
    df = df.dropna(subset=[ftype])
    df = df.drop(['Line', 'Description', '_merge'], axis=1)
    return df

In [10]:
def proc_new_industry(fname, icodes, ftype):
    df = pd.read_csv(fname, header=4, nrows=89, na_values=['...'])
    df = df[df['Line']!='Line']
    df = df.rename(columns={'Unnamed: 1': 'Var'})
    df['Var'] = df['Var'].str.strip()
    
    # We need to modify the descriptions a bit to get them to match up
    recode = {'Publishing industries, except internet (includes software)':'Publishing industries (includes software)',
          'Data processing, internet publishing, and other information services':'Information and data processing services'}
    df['Var'] = df['Var'].replace(recode)
    
    
    df = pd.merge(left=icodes, right=df, left_on='Description', right_on='Var', how='outer', indicator=True)

    # If the industry does not have a code (RD/ITP/ITU) then we do not need it.
    df = df.dropna(subset=[ftype])
    df = df.drop(['Line', 'Description', '_merge'], axis=1)
    return df

In [11]:
def load_industry_data(fname_o, fname_n, icodes):
    df_o = proc_old_industry(fname_o, icodes, 'RD')
    df_n = proc_old_industry(fname_n, icodes, 'RD')

    if (df_o['1997']/df_n['1997']).unique() != 1:
        print('There is a problem matching up the datasets.')
        return '-1'
    
    df_n = df_n.drop('1997', axis=1)
    df = pd.merge(left=df_n, right=df_o, on=['RD', 'NRD', 'ITU', 'NITU', 'ITP', 'NITP', 'IED Code', 'Var']).sort_index(axis=1).set_index(['RD', 'NRD', 'ITU', 'NITU', 'ITP', 'NITP', 'IED Code', 'Var'])
    df.columns = [int(c) for c in df.columns]
    df.sort_index(inplace=True)
    return df


## Nominal value added by industry

1. 1982-1997 are from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=147&step=2&isuri=12).
2. 1997-2016 are from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=150&step=2&isuri=1&categories=gdpxind ).

### Adjusting value added

We need to adjust some of the industry accounts to reconcile themn with the NIPA. First, load the NIPA data, then adjust the industry data. 

1. From real estate, subtract NIPA housing services (NIPA T1.3.5, L11)
2. Subtract the difference between value added for households and institutions (NIPA T1.3.5, L5) and NIPA housing services (NIPA T1.3.5, L11) from education, health care, and social assistance
3. Add government enterprises (AIA lines 94(95) and 97(98) to utilities.

In [15]:
dfo = pd.read_csv('../1-raw-data/va-by-industry-82-97.csv', header=4, nrows=98, na_values=['...'])
dfo = dfo[(dfo['Line']==98) | (dfo['Line']==95)].drop('Line', axis=1).set_index('Unnamed: 1').transpose()
dfo.columns = ['nva_fed', 'nva_sl']

dfn = pd.read_csv('../1-raw-data/va-by-industry-97-16.csv', header=4, nrows=98, na_values=['...'])
dfn = dfn[(dfn['Line']=='94') | (dfn['Line']=='97')].drop('Line', axis=1).set_index('Unnamed: 1').transpose()
dfn.columns = ['nva_fed', 'nva_sl']

df = pd.concat([dfo, dfn.iloc[1:, :]])
df.index = df.index.astype(int)
df.head(1)

,nva_fed,nva_sl
1982,23.8,18.5


In [16]:
vadd = load_industry_data('../1-raw-data/va-by-industry-82-97.csv', '../1-raw-data/va-by-industry-97-16.csv', icodes)
nipa = pd.read_csv('../1-raw-data/NipaDataA.txt', thousands = ',')

# Remove NIPA housing services (1.3.5 line 11) from real estate 
hh11 = nipa[(nipa['%SeriesCode']=='A2009C')&(nipa['Period']>=1982)&(nipa['Period']<=2016)][['Period', 'Value']].set_index('Period')

vadd.loc[(0,0,0,1,0,0,5310,'Real estate')] = vadd.loc[(0,0,0,1,0,0,5310,'Real estate')]-hh11['Value'].transpose()/1000
vadd.loc[(0,0,0,0,0,1,5300,'Real estate and rental and leasing')] = vadd.loc[(0,0,0,0,0,1,5300,'Real estate and rental and leasing')]-hh11['Value'].transpose()/1000
vadd.loc[(0,1,0,0,0,0,np.nan,'Finance, insurance, real estate, rental, and leasing')] = vadd.loc[(0,1,0,0,0,0,np.nan,'Finance, insurance, real estate, rental, and leasing')]-hh11['Value'].transpose()/1000


# Remove NIPA housing services (1.3.5 line 5) 
hh5 = nipa[(nipa['%SeriesCode']=='A193RC')&(nipa['Period']>=1982)&(nipa['Period']<=2016)][['Period', 'Value']].set_index('Period')
hhres = (hh5-hh11)/1000
vadd.loc[(0,1,0,1,0,1,6000,'Educational services, health care, and social assistance')] = vadd.loc[(0,1,0,1,0,1,6000,'Educational services, health care, and social assistance')]- hhres['Value'].transpose()

# Add government enterprises to utilities
vadd.loc[(0,1,0,1,0,1,2200,'Utilities')] = vadd.loc[(0,1,0,1,0,1,2200,'Utilities')]+df['nva_fed']+df['nva_sl'] 

# inds will be the DataFrame we add all our data series to. Set up the indexes. 
inds = pd.DataFrame(vadd.xs(1,level='NRD').sum(),columns = pd.MultiIndex.from_tuples([('nrd', 'nva')]), index=range(1982,2017))
inds[('rd', 'nva')] = vadd.xs(1,level='RD').sum()
inds[('itu', 'nva')] = vadd.xs(1,level='ITU').sum()
inds[('itp', 'nva')] = vadd.xs(1,level='ITP').sum()
inds[('nitu', 'nva')] = vadd.xs(1,level='NITU').sum()
inds[('nitp', 'nva')] = vadd.xs(1,level='NITP').sum()
inds[('pi', 'nva')] = inds[('rd', 'nva')] + inds[('nrd', 'nva')]

inds.head(1)

,nrd,rd,itu,itp,nitu,nitp,pi
,nva,nva,nva,nva,nva,nva,nva
1982,2238.597,329.9,926.8,223.3,1641.697,2345.197,2568.497


## Prices by industry

1. 1982-1997 are from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=147&step=2&isuri=1)
2. 1997-2016 are from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=150&step=2&isuri=1&categories=gdpxind)


In [17]:
price = load_industry_data('../1-raw-data/price-by-industry-82-97.csv', '../1-raw-data/price-by-industry-97-16.csv', icodes)

## Real value added by industry

In [18]:
rvadd = vadd/(price/100)

## Real value added growth by industry

The real value added by industry growth rates have to be constructed from Tornqvist growth rates. 

The tornqvist growth in, say, real value is 
$$ \left(\frac{van_{it}}{\sum van_{it}} + \frac{va_{it-1}}{\sum van_{it-1}}\right)\times\frac{1}{2}\times \Delta_t var_{i}$$

where $van_i$ is nominal value added in $i$ and $var_i$ is real value added.

In [19]:
# Tornqvist index
def tq1(x):
    return (x + x.shift(1))*0.5

def logg(x):
    return (np.log(x) - np.log(x.shift(1)))

In [20]:
def real_value_added(vadd, rvadd, indgroup):
    temp_v = vadd.xs(1,level=indgroup).reset_index(drop=True).transpose()
    temp_rv = rvadd.xs(1,level=indgroup).reset_index(drop=True).transpose()
    
    # Divide nominal value added in each industry by the sum of the industry
    va_share = temp_v.div(temp_v.sum(axis=1), axis=0)
    
    # The first step computes the nominal shares term
    # The second step computes the log growth in real
    temp = va_share.apply(tq1)*temp_rv.apply(logg)
    
    return temp.sum(axis=1)

Apply the real value added calculation to each industry group.

In [21]:
for indgroup in ['RD', 'NRD', 'ITU', 'NITU', 'ITP', 'NITP'] :
    inds[(indgroup.lower(), 'unadj_rva_gr')] = real_value_added(vadd, rvadd, indgroup)

inds.head(1)

,nrd,rd,itu,itp,nitu,nitp,pi,rd,nrd,itu,nitu,itp,nitp
,nva,nva,nva,nva,nva,nva,nva,unadj_rva_gr,unadj_rva_gr,unadj_rva_gr,unadj_rva_gr,unadj_rva_gr,unadj_rva_gr
1982,2238.597,329.9,926.8,223.3,1641.697,2345.197,2568.497,0.0,0.0,0.0,0.0,0.0,0.0


## Gross output by industry

1. 1982-1997 from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=150&step=2&isuri=1&categories=gdpxind)
2. 1997-2016 from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=147&step=2&isuri=1)

In [22]:
go = load_industry_data('../1-raw-data/go-by-industry-82-97.csv', '../1-raw-data/go-by-industry-97-16.csv', icodes)

inds[('rd', 'ngo')] = go.loc[1].sum()
inds[('itu', 'ngo')] = go.xs(1,level='ITU').sum()
inds[('itp', 'ngo')] = go.xs(1,level='ITP').sum()

# Keep total gross output, too
inds[('pi', 'ngo')] = go.xs('Private industries', level='Var').reset_index(drop=True).transpose()

## Labor
1982-1987 is from BEA, NIPA Table 6.5b

The file `industry-classifications-labor.csv` lists an author-created concordance between the BEA industries and the RD/ITU/ITP groups. 

In [23]:
cols = ['Description', 1982, 1983, 1984, 1985, 1986, 1987]
emp = pd.read_csv('../1-raw-data/nipa-T6.5b.csv', header=None, nrows=75, names = cols, skiprows=6)
emp['Description'] = emp['Description'].str.strip()

emp_icodes = pd.read_csv('../1-raw-data/author-created-concordances/industry-classifications-labor.csv')
t = pd.merge(left=emp_icodes, right=emp, on='Description', how='inner').set_index(['RD','ITU','ITP','Description'])

empbea = pd.concat([t.xs(1,level='RD').sum(), t.xs(1,level='ITU').sum(), t.xs(1,level='ITP').sum()], axis=1, keys=['RD', 'ITU', 'ITP'])
empbea.index = empbea.index.astype(int).copy()
empbea.tail(1)

,RD,ITU,ITP
1987,12577,37574,10927


1987-2016 is from the BLS [labor productivity and costs database](https://www.bls.gov/lpc/). We use the "industry productivity" text file. 

In [24]:
bls = pd.read_csv('../1-raw-data/ip.data.1.AllData.txt', sep='\t')
bls.columns = [c.strip() for c in bls.columns] 
bls['series_id'] = bls['series_id'].str.strip()
bls = bls.set_index('series_id')

bls = bls[ (bls['year']>1986) & (bls['year']<2017)]
bls = bls[['year', 'value']]

rd = ['IPUEN325___L200000000', 'IPUEN334___L200000000', 'IPUEN336___L200000000', 'IPUJN5112__L200000000',
      'IPUMN5415__L200000000', 'IPUMN5417__L200000000']

itu = ['IPUEN315___L200000000', 'IPUEN333___L200000000', 'IPUEN335___L200000000', 'IPUEN336___L200000000',
       'IPUEN337___L200000000', 'IPUEN339___L200000000', 'IPUGN42____L200000000', 'IPUHN44_45_L200000000',
       'IPUJN511___L200000000', 'IPUJN512___L200000000', 'IPUJN515___L200000000', 'IPUJN517___L200000000',
       'IPUJN518___L200000000', 'IPUJN519___L200000000', 'IPULN532___L200000000', 'IPUMN5417__L200000000']

itp = ['IPUEN334___L200000000', 'IPUJN511___L200000000', 'IPUJN512___L200000000', 'IPUJN515___L200000000',
       'IPUJN517___L200000000', 'IPUJN518___L200000000', 'IPUJN519___L200000000', 'IPUMN5415__L200000000']

# The set() removes the duplicates
t = bls.loc[list(set(itp+itu+rd))].reset_index()
t = t.set_index(['year', 'series_id']).unstack().droplevel(level=0, axis=1)

empbls = pd.concat([t[rd].sum(axis=1),t[itu].sum(axis=1),t[itp].sum(axis=1)], axis=1, keys=['RD', 'ITU', 'ITP'])
empbls.head(1)

,RD,ITU,ITP
year,,,
1987,11829.897,51462.836,9348.43


Splice the two series together. Add the aggregates to `inds`. 

In [25]:
f = empbls.loc[1987] / empbea.loc[1987]
emp = pd.concat([empbea * f, empbls.loc[1988:]])

inds[('rd', 'emp')] = emp['RD']
inds[('itu', 'emp')] = emp['ITU']
inds[('itp', 'emp')] = emp['ITP']

### Non-XX intensive firms

Non-RD/Non-ITU/Non-ITP labor is total labor minus RD/ITU/ITP labor. Total private secotr labor is from https://www.bls.gov/lpc/ under "LPC tables and charts" and then "Total U.S. Economy - all worker - November 4, 2021".

In [26]:
hours = pd.read_excel('../1-raw-data/us_total_hrs_emp.xlsx', sheet_name='Hours', header=None, skiprows=8, usecols=[0,8,9],
                     parse_dates=[0], index_col=0, names=['date','farm', 'nonfarm_biz'])

hours['emp'] = (hours['farm'] + hours['nonfarm_biz'])*1000
hours = hours.resample('Y').mean()
hours.index = hours.index.year

inds[('total', 'emp')] = hours.loc[1982:2016, 'emp']
inds[('nrd', 'emp')] = inds[('total', 'emp')] - inds[('rd', 'emp')]
inds[('nitu', 'emp')] = inds[('total', 'emp')] - inds[('itu', 'emp')]
inds[('nitp', 'emp')] = inds[('total', 'emp')] - inds[('itp', 'emp')]

/var/folders/8y/w4dpfrqs7gv35k28lrpw_xy00000gn/T/ipykernel_6350/2378380529.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hours = pd.read_excel('../1-raw-data/us_total_hrs_emp.xlsx', sheet_name='Hours', header=None, skiprows=8, usecols=[0,8,9],


## Productivity

Here we compute (all growth rates are log differences)
1. nominal value-added growth 
2. employment growth 
3. price index growth (difference between nominal and real value added), which we need to make the adjustments later
4. unadjusted productivity growth (gr), and cumulative growth (cgr)

In [27]:
for indgroup in ['rd', 'nrd', 'itu', 'nitu', 'itp', 'nitp']:
    inds[(indgroup, 'nva_gr')]         = pd.DataFrame(inds[(indgroup, 'nva')]).apply(logg)
    inds[(indgroup, 'emp_gr')]         = pd.DataFrame(inds[(indgroup, 'emp')]).apply(logg)
    inds[(indgroup, 'p_gr')]           = inds[(indgroup, 'nva_gr')] - inds[(indgroup, 'unadj_rva_gr')]
    inds[(indgroup, 'prod_unadj_gr')]  = inds[(indgroup, 'unadj_rva_gr')] - inds[(indgroup, 'emp_gr')]
    inds[(indgroup, 'prod_unadj_cgr')] = inds[(indgroup, 'prod_unadj_gr')].cumsum()
    inds.loc[1982, (indgroup, 'prod_unadj_cgr')] = 0

inds = inds.sort_index(axis=1)

In [28]:
inds.to_csv('../3-intermediate-files/industry.csv')
inds

itp                                                 
               emp    emp_gr     ngo     nva    nva_gr      p_gr   
1982   7447.431422       NaN   408.8   223.3       NaN       NaN  \
1983   7642.493382  0.025855   451.0   254.5  0.130785  0.023453   
1984   8332.910058  0.086489   511.0   281.5  0.100832  0.039490   
1985   8699.934536  0.043103   539.4   307.0  0.086715  0.002143   
1986   8966.861429  0.030220   567.6   319.8  0.040848  0.006365   
1987   9348.430000  0.041673   602.3   342.7  0.069160 -0.003910   
1988   9556.255000  0.021987   652.3   365.5  0.064411  0.001953   
1989   9732.414000  0.018266   685.4   393.6  0.074069  0.010766   
1990   9680.534000 -0.005345   722.5   415.1  0.053184  0.003782   
1991   9366.187000 -0.033011   745.2   431.1  0.037821  0.017588   
1992   9258.256000 -0.011590   783.1   457.7  0.059874 -0.009080   
1993   9451.958000  0.020706   853.0   487.4  0.062871 -0.006157   
1994   9738.380000  0.029853   938.3   527.4  0.078874 -0.007991   
1995  10208.214000  0.047118  1059.7   567.0  0.072400 -0.039597   
1996  10689.904000  0.046107  1163.8   619.1  0.087908 -0.042238   
1997  11389.015000  0.063350  1315.6   664.9  0.071370 -0.032982   
1998  11958.762000  0.048815  1426.7   715.3  0.073065 -0.077003   
1999  12491.298000  0.043568  1585.9   775.0  0.080161 -0.060475   
2000  13084.018000  0.046359  1767.9   810.6  0.044912 -0.059744   
2001  12830.305000 -0.019582  1705.6   794.0 -0.020691 -0.053727   
2002  11588.625000 -0.101786  1635.0   833.6  0.048670 -0.025332   
2003  10836.724000 -0.067083  1634.5   874.1  0.047441 -0.041419   
2004  10679.063000 -0.014656  1684.4   944.7  0.077673 -0.042623   
2005  10657.192000 -0.002050  1757.3   988.0  0.044815 -0.042334   
2006  10749.359000  0.008611  1840.3  1024.9  0.036668 -0.040870   
2007  10875.458000  0.011663  1921.9  1103.0  0.073439 -0.035717   
2008  10907.197000  0.002914  1960.1  1157.4  0.048142 -0.035054   
2009  10325.259000 -0.054830  1849.6  1133.8 -0.020601 -0.015479   
2010  10083.789000 -0.023664  1932.7  1198.0  0.055079 -0.025213   
2011  10087.605000  0.000378  2017.3  1225.6  0.022777 -0.011108   
2012  10210.279000  0.012088  2090.9  1256.7  0.025059 -0.006110   
2013  10314.876000  0.010192  2148.9  1331.2  0.057592 -0.004000   
2014  10506.527000  0.018410  2238.3  1362.0  0.022873 -0.012392   
2015  10788.396000  0.026474  2307.8  1460.2  0.069619 -0.021995   
2016  10943.326000  0.014259  2436.8  1543.3  0.055350 -0.019543   

                                                         itu  ...   
     prod_unadj_cgr prod_unadj_gr unadj_rva_gr           emp  ...   
1982       0.000000           NaN     0.000000  43297.045075  ...  \
1983       0.081477      0.081477     0.107332  43817.508099  ...   
1984       0.056330     -0.025148     0.061341  46981.375432  ...   
1985       0.097799      0.041469     0.084572  48779.712145  ...   
1986       0.102062      0.004263     0.034483  49815.159636  ...   
1987       0.133459      0.031397     0.073070  51462.836000  ...   
1988       0.173930      0.040471     0.062458  52850.376000  ...   
1989       0.218966      0.045037     0.063303  53835.663000  ...   
1990       0.273714      0.054747     0.049402  53379.087000  ...   
1991       0.326957      0.053244     0.020233  52112.153000  ...   
1992       0.407501      0.080544     0.068954  51741.884000  ...   
1993       0.455824      0.048322     0.069029  52506.966000  ...   
1994       0.512836      0.057012     0.086865  54060.025000  ...   
1995       0.577715      0.064879     0.111997  55139.058000  ...   
1996       0.661753      0.084039     0.130146  55658.312000  ...   
1997       0.702756      0.041002     0.104352  56856.786000  ...   
1998       0.804009      0.101253     0.150068  57448.992000  ...   
1999       0.901077      0.097068     0.140636  58273.259000  ...   
2000       0.959373      0.058296     0.104655  58854.287000  ...   
2001       1.011991      0.052617     0.033036  5